In [31]:
import sys
import os.path
import os
import math
import datetime, time
import numpy as np
from sklearn import preprocessing, model_selection
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier
#from scipy.misc import imread
from matplotlib.pyplot import imread
import cv2
#from xgboost import XGBClassifier
#from lightgbm import LGBMClassifier
from sklearn.pipeline import Pipeline
from PIL import Image

import lfw
from sklearn import metrics
from scipy.optimize import brentq
from scipy import interpolate

# Data

In [2]:
DATA_PATH='D:/datasets/lfw_ytf/'
DATASET_PATH=DATA_PATH+'lfw_mtcnn_aligned'
#DATASET_PATH=DATA_PATH+'lfw_mtcnn'

In [5]:
img_extensions=['.jpg','.jpeg','.png']
def is_image(path):
    _, file_extension = os.path.splitext(path)
    return file_extension.lower() in img_extensions

def get_files(db_dir):
    return [[d,os.path.join(d,f)] for d in next(os.walk(db_dir))[1] for f in sorted(next(os.walk(os.path.join(db_dir,d)))[2]) if not f.startswith(".") and is_image(f)]

In [25]:
def get_single_image_per_class_cv(y, n_splits=10,random_state=0):
    res_cv=[]
    inds = np.arange(len(y))
    np.random.seed(random_state)
    for _ in range(n_splits):
        inds_train, inds_test = [], []

        for lbl in np.unique(y):
            tmp_inds = inds[y == lbl]
            np.random.shuffle(tmp_inds)
            last_ind=1
            #last_ind=math.ceil(len(tmp_inds)/2)
            if last_ind==0 and len(tmp_inds)>0:
                last_ind=1
            inds_train.extend(tmp_inds[:last_ind])
            inds_test.extend(tmp_inds[last_ind:])
            
        inds_train = np.array(inds_train)
        inds_test = np.array(inds_test)
    
        res_cv.append((inds_train, inds_test))
    return res_cv

def classifier_tester(classifier,x,y):
    sss=get_single_image_per_class_cv(y)
    #sss=model_selection.StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=0)
    scores=model_selection.cross_validate(classifier,x, y, scoring='accuracy',cv=sss)
    acc=scores['test_score']
    print('accuracies=',acc*100)
    print('total acc=',round(acc.mean()*100,2),round(acc.std()*100,2))
    print('test time=',scores['score_time'])

## Recognition 

In [20]:
if False:
    dirs_and_files=np.array(get_files(DATASET_PATH))
else: #LFW and YTF concatenation
    subjects = (line.rstrip('\n') for line in open('lfw_ytf_classes.txt'))
    dirs_and_files=np.array([[d,os.path.join(d,f)] for d in subjects for f in sorted(next(os.walk(os.path.join(DATASET_PATH,d)))[2]) if is_image(f)])

dirs=dirs_and_files[:,0]
files=dirs_and_files[:,1]

label_enc=preprocessing.LabelEncoder()
label_enc.fit(dirs)
y=label_enc.transform(dirs)

y_l=list(y)
indices=[i for i,el in enumerate(y_l) if y_l.count(el) > 1]
y=y[indices]
label_enc=preprocessing.LabelEncoder()
label_enc.fit(y)
y=label_enc.transform(y)
print('after loading: num_classes=',len(label_enc.classes_))

after loading: num_classes= 596


In [30]:
pca_components=128 #256
classifiers=[]
classifiers.append(['k-NN+PCA',Pipeline(steps=[('pca', PCA(n_components=pca_components)), ('classifier', KNeighborsClassifier(n_neighbors=1,p=2))])])
classifiers.append(['k-NN',KNeighborsClassifier(n_neighbors=1,p=2)])
classifiers.append(['linear svm',LinearSVC()])

def test_faceID():
    for cls_name,classifier in classifiers:
        print(cls_name)
        classifier_tester(classifier,X_norm,y)
        print()

## Verification

In [32]:
pairs = lfw.read_pairs(DATA_PATH+'pairs.txt')
#print(pairs.shape,pairs[:5],pairs[-5:])
# Get the paths for the corresponding images
paths, actual_issame = lfw.get_paths(DATASET_PATH, pairs)
#print(len(paths), len(actual_issame), paths[:5], actual_issame[:5], paths[-5:], actual_issame[-5:])

embedding_indices=np.zeros(len(paths), dtype=int)
filepaths={}
for i,p in enumerate(paths):
    if p not in filepaths:
        filepaths[p]=len(filepaths)
    embedding_indices[i]=filepaths[p]
#print(len(filepaths),len(embedding_indices), len(actual_issame), embedding_indices[:5], actual_issame[:5], embedding_indices[-5:], actual_issame[-5:])

files4verification=[None]*len(filepaths)
for f,i in filepaths.items():
    files4verification[i]=f


D:\src_code\mobile\mobile-face-recognition\src\lfw.py:193: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(pairs)


In [46]:
def print_verification_results(X):
    nrof_embeddings = len(actual_issame)*2  # nrof_pairs * nrof_images_per_pair
    embedding_size=X.shape[1]
    embeddings = np.zeros((nrof_embeddings, embedding_size))
    for i,emb_ind in enumerate(embedding_indices):
        embeddings[i,:]=X[emb_ind]

    tpr, fpr, accuracy, val, val_std, far = lfw.evaluate(embeddings, actual_issame, nrof_folds=10, distance_metric=0)

    print('Accuracy: %2.5f+-%2.5f' % (np.mean(accuracy), np.std(accuracy)))
    print('Validation rate: %2.5f+-%2.5f @ FAR=%2.5f' % (val, val_std, far))

    auc = metrics.auc(fpr, tpr)
    print('Area Under Curve (AUC): %1.5f' % auc)
    eer = brentq(lambda x: 1. - x - interpolate.interp1d(fpr, tpr)(x), 0., 1.)
    print('Equal Error Rate (EER): %1.3f' % eer)

# PyTorch

In [40]:
import torch
import torch.nn as nn
use_cuda = torch.cuda.is_available()
device = 'cuda' if use_cuda else 'cpu'
print(device)

cuda


## Our models 

In [4]:
import json
sys.path.append("../once-for-all")
from ofa.imagenet_classification.networks.mobilenet_v3 import MobileNetV3
from ofa.imagenet_classification.elastic_nn.networks import OFAMobileNetV3

def get_model_artifacts(net_folder,model_name):
    config_filepath = os.path.join(net_folder, model_name,f"{model_name}.config")
    test_pt_filepath = os.path.join(net_folder, model_name,f"{model_name}.pth")
    model_config = json.load(open(config_filepath, 'r'))
    test_checkpoint = torch.load(test_pt_filepath, map_location=torch.device('cpu'))
    filter_end_fn = lambda x : not x.endswith('total_ops') and not x.endswith('total_params')
    filter_start_fn = lambda x : not x.startswith('total_ops') and not x.startswith('total_params')
    filtered_state_dict = {key:value for key,value in test_checkpoint['state_dict'].items() if filter_start_fn(key) and filter_end_fn(key)}
    model=MobileNetV3.build_from_config(model_config)
    model.load_state_dict(filtered_state_dict)
    #torch.save(model,model_name+'.pt')
    return model

In [50]:
def get_my_model(model_name):
    print(model_name)
    if model_name=='SuperNet':
        cnn_model = OFAMobileNetV3(n_classes=9131,dropout_rate=0.,width_mult=1.2, ks_list= [3, 5, 7],expand_ratio_list=[3, 4, 6],depth_list=[2, 3, 4])
        state_dict = torch.load('../models/ofa_mbv3_model_best_new.pth.tar', map_location="cpu")["state_dict"]
        cnn_model.load_state_dict(state_dict)
        #print(cnn_model)
    else:
        cnn_model=get_model_artifacts('../models/ofa_subnets',model_name)
        #cnn_model=get_model_artifacts('D:/src_code/DNN_models/ofa_subnets',model_name)
    cnn_model.classifier=torch.nn.Identity()
    cnn_model=cnn_model.to(device)
    cnn_model.eval()
    return cnn_model

model_names=[
    'SuperNet',

    'subnet_device_865_acc_98.3_lut_12.1ms_w12_d234_nac_gbdt',
    'subnet_device_865_acc_97.8_lut_9.15ms_w12_d234_nac_gbdt',
    'subnet_device_765_acc_98.3_lut_34.6ms_w12_d234_nac_gbdt',
    'subnet_device_765_acc_97.7_lut_24.45ms_w12_d234_nac_gbdt',

    #'subnet_device_865_acc_97.9_lut_12.1ms_w12_d234_reg_predictor',
    #'subnet_device_865_acc_97.7_lut_9.15ms_w12_d234_reg_predictor',
    #'subnet_device_765_acc_97.9_lut_34.6ms_w12_d234_reg_predictor',
    #'subnet_device_765_acc_97.7_lut_24.45ms_w12_d234_reg_predictor',

    #'subnet_device_865_acc_98.3_lut_12.1ms_w12_d234_reg_gbdt',
    #'subnet_device_865_acc_97.8_lut_9.15ms_w12_d234_reg_gbdt',
    #'subnet_device_765_acc_98.4_lut_34.6ms_w12_d234_reg_gbdt',
    #'subnet_device_765_acc_97.7_lut_24.45ms_w12_d234_reg_gbdt',

]

---------------

In [37]:
def extract_torch_features(model, files,crop_center=False, standard_preprocessing=True):
    if standard_preprocessing:
        from torchvision import transforms
        test_transforms = transforms.Compose(
            [
                transforms.Resize((224,224)),
                #transforms.Resize((260,260)),
                #transforms.Resize((160,160)),
                #transforms.Resize((196,196)),
                #transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                             std=[0.229, 0.224, 0.225])
            ]
        )
    else:
        mean_bgr = np.array([91.4953, 103.8827, 131.0912])
        def test_transforms(img):
            img = np.array(img.resize((224,224),Image.BILINEAR))
            img = img[:, :, ::-1]  # RGB -> BGR
            img = img.astype(np.float32)
            img -= mean_bgr
            img = img.transpose(2, 0, 1)  # C x H x W
            img = torch.from_numpy(img).float()
            return img
    X=[]
    imgs=[]
    for filepath in files:
        img = Image.open(os.path.join(DATASET_PATH,filepath))
        if crop_center:
            orig_w,orig_h=250,250
            img = img.resize((orig_w,orig_h),Image.BILINEAR)
            w1,h1=224,224#128,128
            dw=(orig_w-w1)/2
            dh=(orig_h-h1)/2
            box = (dw, dh, orig_w-dw, orig_h-dh)
            img = img.crop(box)
        #img = img.resize((224,224),Image.BILINEAR)
        img_tensor = test_transforms(img)
            
        if img.size:
            imgs.append(img_tensor)
            if len(imgs)>=8: #1:#8:
                scores = model(torch.stack(imgs, dim=0).to(device))
                scores=scores.data.cpu().numpy()
                #print(scores.shape)
                if len(X)==0:
                    X=scores
                else:
                    X=np.concatenate((X,scores),axis=0)
                
                imgs=[]

    if len(imgs)>0:        
        scores = model(torch.stack(imgs, dim=0).to(device))
        scores=scores.data.cpu().numpy()
        #print(scores.shape)
        if len(X)==0:
            X=scores
        else:
            X=np.concatenate((X,scores),axis=0)
    
    return X

In [51]:
for model_name in model_names:
    cnn_model=get_my_model(model_name)
    
    start_time = time.time()
    X=extract_torch_features(cnn_model,files)
    print('--- %s seconds ---' % (time.time() - start_time))

    X_norm=preprocessing.normalize(X,norm='l2')
    X_norm=X_norm[indices,:]
    print('after loading: X shape:',X.shape,' X_norm shape:',X_norm.shape)
    
    print('Recognition:')
    test_faceID()

SuperNet
--- 31.18065595626831 seconds ---
after loading: X shape: (3739, 1536)  X_norm shape: (3739, 1536)
Recognition:
k-NN+PCA
accuracies= [99.1091314  99.36366529 96.72287623 99.1091314  98.98186446 99.0136812
 99.0136812  99.20458161 98.72733058 99.33184855]
total acc= 98.86 0.73
test time= [0.0977385  0.10076785 0.10571551 0.09678173 0.09877229 0.09574366
 0.0997715  0.09478331 0.11269784 0.09478498]

k-NN
accuracies= [99.42729876 99.42729876 96.02290805 99.17276487 99.36366529 99.14094814
 99.14094814 99.36366529 99.07731467 99.58638244]
total acc= 98.97 1.0
test time= [0.15256619 0.16855097 0.16256428 0.15557933 0.16060805 0.14960766
 0.15554214 0.17253923 0.1496408  0.15753984]

linear svm
accuracies= [99.36366529 99.45911549 97.58192809 99.33184855 99.33184855 99.30003182
 99.23639835 99.33184855 99.23639835 99.42729876]
total acc= 99.16 0.53
test time= [0.05688596 0.05487347 0.05385447 0.05387783 0.05484939 0.05385566
 0.05386043 0.05389476 0.06085896 0.07177281]

subnet_dev

In [52]:
print('Verification')
for model_name in model_names:
    cnn_model=get_my_model(model_name)
    start_time = time.time()
    X=extract_torch_features(cnn_model,files4verification)
    print('--- %s seconds ---' % (time.time() - start_time))

    print(X.shape)
    X=preprocessing.normalize(X,norm='l2')

    print_verification_results(X)
    print()

Verification
SuperNet
--- 67.26128721237183 seconds ---
(7701, 1536)
Accuracy: 0.99317+-0.00337
Validation rate: 0.98000+-0.01366 @ FAR=0.00133
Area Under Curve (AUC): 0.99978
Equal Error Rate (EER): 0.006
subnet_device_865_acc_98.3_lut_12.1ms_w12_d234_nac_gbdt
--- 34.714967012405396 seconds ---
(7701, 1536)
Accuracy: 0.99217+-0.00350
Validation rate: 0.98133+-0.01194 @ FAR=0.00100
Area Under Curve (AUC): 0.99970
Equal Error Rate (EER): 0.008
subnet_device_865_acc_97.8_lut_9.15ms_w12_d234_nac_gbdt
--- 30.85317611694336 seconds ---
(7701, 1536)
Accuracy: 0.99200+-0.00414
Validation rate: 0.97700+-0.01286 @ FAR=0.00100
Area Under Curve (AUC): 0.99955
Equal Error Rate (EER): 0.010
subnet_device_765_acc_98.3_lut_34.6ms_w12_d234_nac_gbdt
--- 35.97784209251404 seconds ---
(7701, 1536)
Accuracy: 0.99300+-0.00400
Validation rate: 0.98133+-0.01352 @ FAR=0.00133
Area Under Curve (AUC): 0.99974
Equal Error Rate (EER): 0.007
subnet_device_765_acc_97.7_lut_24.45ms_w12_d234_nac_gbdt
--- 30.631898641

## SeNet (VggFace2) 

In [38]:
cnn_model = torch.load('D:/src_code/DNN_models/vggface_senet50_1.pt')
cnn_model=cnn_model.to(device)
cnn_model.eval()

SENet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1)

In [39]:
start_time = time.time()
X=extract_torch_features(cnn_model,files, standard_preprocessing=False)
print('--- %s seconds ---' % (time.time() - start_time))

X_norm=preprocessing.normalize(X,norm='l2')
X_norm=X_norm[indices,:]
print('after loading: X shape:',X.shape,' X_norm shape:',X_norm.shape)

C:\Users\avsavchenko\AppData\Local\Temp\ipykernel_18884\2271549186.py:19: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  img = np.array(img.resize((224,224),Image.BILINEAR))


--- 42.60752272605896 seconds ---
after loading: X shape: (3739, 2048)  X_norm shape: (3739, 2048)


In [41]:
print('Recognition:')
test_faceID()

Recognition:
k-NN+PCA
accuracies= [98.95004773 98.79096405 84.66433344 98.02736239 98.63188037 98.25007954
 98.79096405 98.85459752 98.21826281 98.88641425]
total acc= 97.21 4.19
test time= [0.10276651 0.10775042 0.10376048 0.10375953 0.10376334 0.10475445
 0.10176659 0.10177088 0.09973216 0.10176492]

k-NN
accuracies= [98.91823099 98.91823099 84.6325167  98.1228126  98.50461343 98.25007954
 98.82278078 98.82278078 98.37734648 98.91823099]
total acc= 97.23 4.21
test time= [0.19647241 0.19351745 0.20142484 0.20349264 0.19643807 0.20541239
 0.19843364 0.20046544 0.18350315 0.18451214]

linear svm
accuracies= [98.88641425 98.91823099 87.97327394 98.40916322 98.5682469  98.53643016
 98.88641425 98.88641425 98.69551384 99.0136812 ]
total acc= 97.68 3.24
test time= [0.0748024  0.07675815 0.07280827 0.0828011  0.07882667 0.07879925
 0.07081342 0.06882143 0.06784153 0.07679391]



In [49]:
start_time = time.time()
X=extract_torch_features(cnn_model,files4verification, standard_preprocessing=False)
print('--- %s seconds ---' % (time.time() - start_time))
print('Verification')

print(X.shape)
X=preprocessing.normalize(X,norm='l2')

print_verification_results(X)

C:\Users\avsavchenko\AppData\Local\Temp\ipykernel_18884\2271549186.py:19: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  img = np.array(img.resize((224,224),Image.BILINEAR))


--- 78.83677887916565 seconds ---
Verification
(7701, 2048)
Accuracy: 0.99367+-0.00371
Validation rate: 0.97833+-0.01276 @ FAR=0.00100
Area Under Curve (AUC): 0.99964
Equal Error Rate (EER): 0.006


# Insightface

In [53]:
import insightface
#model_path='C:/Users/avsavchenko/.insightface/models/buffalo_l/w600k_r50.onnx'
model_path='D:/src_code/DNN_models/age_gender/insightface/models/vgg2_r50_pfc.onnx'
cnn_model = insightface.model_zoo.get_model(model_path,providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
cnn_model.prepare(ctx_id=0)
print(cnn_model)

d:\program files\python38\lib\site-packages\onnxruntime\capi\onnxruntime_inference_collection.py:55: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CPUExecutionProvider'
  warnings.warn("Specified provider '{}' is not in available provider names."


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}


In [54]:
def extract_insightface_features(model,img_filepath):
    img = cv2.imread(img_filepath)
    img=img[:, :, ::-1] #bgr2rgb
    embeddings = model.get_feat(img)
    if embeddings is None:
        print(img_filepath)
    #print(embeddings.shape)
    return embeddings[0]

In [55]:
start_time = time.time()
X=np.array([extract_insightface_features(cnn_model,os.path.join(DATASET_PATH,filepath)) for filepath in files])
print('--- %s seconds ---' % (time.time() - start_time))

X_norm=preprocessing.normalize(X,norm='l2')
X_norm=X_norm[indices,:]
print('after loading: X shape:',X.shape,' X_norm shape:',X_norm.shape)

print('Recognition:')
test_faceID()

--- 523.4497258663177 seconds ---
after loading: X shape: (3739, 512)  X_norm shape: (3739, 512)
Recognition:
k-NN+PCA
accuracies= [99.17276487 98.98186446 98.63188037 99.0136812  98.95004773 99.14094814
 99.1091314  98.91823099 99.14094814 99.26821508]
total acc= 99.03 0.17
test time= [0.09482813 0.09382486 0.09677649 0.09179568 0.09578252 0.09574437
 0.0988121  0.09225655 0.0977385  0.09474683]

k-NN
accuracies= [99.30003182 99.33184855 98.88641425 99.33184855 99.20458161 99.14094814
 99.1091314  99.17276487 99.30003182 99.49093223]
total acc= 99.23 0.16
test time= [0.10568261 0.11773086 0.10571599 0.10622478 0.107301   0.10942411
 0.10870934 0.10779071 0.10775208 0.10622168]

linear svm
accuracies= [99.26821508 99.42729876 98.69551384 99.26821508 99.26821508 99.17276487
 99.07731467 99.23639835 99.30003182 99.5545657 ]
total acc= 99.23 0.22
test time= [0.02994299 0.02800465 0.02993798 0.02991915 0.02692151 0.025038
 0.03490615 0.03191566 0.02501273 0.02595019]



In [56]:
start_time = time.time()
X=np.array([extract_insightface_features(cnn_model,os.path.join(DATASET_PATH,filepath)) for filepath in files4verification])
print('--- %s seconds ---' % (time.time() - start_time))
print('Verification')

print(X.shape)
X=preprocessing.normalize(X,norm='l2')

print_verification_results(X)

--- 1063.6578254699707 seconds ---
Verification
(7701, 512)
Accuracy: 0.99217+-0.00373
Validation rate: 0.98567+-0.00978 @ FAR=0.00100
Area Under Curve (AUC): 0.99929
Equal Error Rate (EER): 0.006


# Tensorflow (FaceNet)

In [57]:
import tensorflow as tf
import tensorflow.keras.backend as K 

print(tf.__version__)
from tensorflow.compat.v1.keras.backend import set_session 
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess=tf.compat.v1.Session(config=config)
set_session(sess)

os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

2.4.1


In [58]:
def load_graph(frozen_graph_filename, prefix=''):
    with tf.io.gfile.GFile(frozen_graph_filename, 'rb') as f:
        graph_def = tf.compat.v1.GraphDef()
        graph_def.ParseFromString(f.read())

    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def, name=prefix)
    return graph

class TensorFlowInference:
    def __init__(self,frozen_graph_filename,input_tensor,output_tensor,learning_phase_tensor=None, convert2BGR=True, imageNetUtilsMean=True,additional_input_value=0):
        graph = load_graph(frozen_graph_filename,'')
        print([n.name for n in graph.as_graph_def().node if 'input' in n.name])
        
        graph_op_list=list(graph.get_operations())
        print([n.name for n in graph_op_list if 'keras_learning' in n.name])
        
        self.tf_sess=tf.compat.v1.Session(graph=graph)
        
        self.tf_input_image = graph.get_tensor_by_name(input_tensor)
        print('tf_input_image=',self.tf_input_image)
        self.tf_output_features = graph.get_tensor_by_name(output_tensor)
        print('tf_output_features=',self.tf_output_features)
        self.tf_learning_phase = graph.get_tensor_by_name(learning_phase_tensor) if learning_phase_tensor else None;
        print('tf_learning_phase=',self.tf_learning_phase)
        if self.tf_input_image.shape.dims is None:
            w=h=160
        else:
            _,w,h,_=self.tf_input_image.shape
        self.w,self.h=int(w),int(h)
        print ('input w,h',self.w,self.h,' output shape:',self.tf_output_features.shape,'convert2BGR:',convert2BGR, 'imageNetUtilsMean:',imageNetUtilsMean)
        #for n in graph.as_graph_def().node:
        #    print(n.name, n.op)
        #sys.exit(0)

        self.convert2BGR=convert2BGR
        self.imageNetUtilsMean=imageNetUtilsMean
        self.additional_input_value=additional_input_value

    def preprocess_image(self,img_filepath,crop_center):
        if crop_center:
            orig_w,orig_h=250,250
            img = imread(img_filepath)#, mode='RGB')
            #img = misc.imresize(img, (orig_w,orig_h), interp='bilinear')
            img = np.array(Image.fromarray(img).resize((orig_w,orig_h),resample=Image.BILINEAR))
            w1,h1=128,128
            dw=(orig_w-w1)//2
            dh=(orig_h-h1)//2
            box = (dw, dh, orig_w-dw, orig_h-dh)
            img=img[dh:-dh,dw:-dw]
        else:
            img = imread(img_filepath)#, mode='RGB')
        
        #x = misc.imresize(img, (self.w,self.h), interp='bilinear').astype(float)
        x = np.array(Image.fromarray(img).resize((self.w,self.h),resample=Image.BILINEAR)).astype(float)
        
        if self.convert2BGR:
            # 'RGB'->'BGR'
            x = x[..., ::-1]
            # Zero-center by mean pixel
            if self.imageNetUtilsMean: #imagenet.utils caffe
                x[..., 0] -= 103.939
                x[..., 1] -= 116.779
                x[..., 2] -= 123.68
            else: #vggface-2
                x[..., 0] -= 91.4953
                x[..., 1] -= 103.8827
                x[..., 2] -= 131.0912
        else:
            #x=(x-127.5)/128.0
            x /= 127.5
            x -= 1.
            #x=x/128.0-1.0
        return x
        
    def extract_features(self,img_filepath,crop_center=False):
        x=self.preprocess_image(img_filepath,crop_center)
        x = np.expand_dims(x, axis=0)
        feed_dict={self.tf_input_image: x}
        if self.tf_learning_phase is not None:
            feed_dict[self.tf_learning_phase]=self.additional_input_value
        preds = self.tf_sess.run(self.tf_output_features, feed_dict=feed_dict).reshape(-1)
        #preds = self.tf_sess.run(self.tf_output_features, feed_dict=feed_dict).mean(axis=(0,1,2)).reshape(-1)
        return preds
    
    def close_session(self):
        self.tf_sess.close()


In [59]:
#cnn_model=TensorFlowInference('D:/src_code/HSE_FaceRec_tf/age_gender_identity/age_gender_tf2_new-01-0.14-0.92.pb',input_tensor='input_1:0',output_tensor='global_pooling/Mean:0',convert2BGR=True, imageNetUtilsMean=True)

#FaceNet
cnn_model=TensorFlowInference('D:/src_code/DNN_models/facenet_inceptionresnet/20180402-114759.pb',input_tensor='input:0',output_tensor='embeddings:0',learning_phase_tensor='phase_train:0',convert2BGR=False) #embeddings, InceptionResnetV1/Repeat_2/block8_5/Relu, InceptionResnetV1/Repeat_1/block17_10/Relu

['input']
[]
tf_input_image= Tensor("input:0", dtype=float32)
tf_output_features= Tensor("embeddings:0", shape=(None, 512), dtype=float32)
tf_learning_phase= Tensor("phase_train:0", dtype=bool)
input w,h 160 160  output shape: (None, 512) convert2BGR: False imageNetUtilsMean: True


In [60]:
crop_center=False
start_time = time.time()
X=np.array([cnn_model.extract_features(os.path.join(DATASET_PATH,filepath),crop_center=crop_center) for filepath in files])    
print('--- %s seconds ---' % (time.time() - start_time))

X_norm=preprocessing.normalize(X,norm='l2')
X_norm=X_norm[indices,:]
print('after loading: X shape:',X.shape,' X_norm shape:',X_norm.shape)

print('Recognition:')
test_faceID()

C:\Users\avsavchenko\AppData\Local\Temp\ipykernel_18884\1513849870.py:55: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  x = np.array(Image.fromarray(img).resize((self.w,self.h),resample=Image.BILINEAR)).astype(float)


--- 120.72068953514099 seconds ---
after loading: X shape: (3739, 512)  X_norm shape: (3739, 512)
Recognition:
k-NN+PCA
accuracies= [97.39102768 97.74101177 85.93700286 96.85014318 96.7865097  96.75469297
 97.6773783  97.42284442 96.97741012 97.6773783 ]
total acc= 96.12 3.41
test time= [0.08680582 0.08481026 0.08580971 0.08681345 0.088763   0.08581114
 0.09578013 0.08676887 0.08680677 0.08776951]

k-NN
accuracies= [97.39102768 97.74101177 85.93700286 96.85014318 96.7865097  96.75469297
 97.6773783  97.42284442 96.97741012 97.6773783 ]
total acc= 96.12 3.41
test time= [0.09674358 0.10072756 0.10169077 0.10475898 0.09973192 0.10172582
 0.10268569 0.10375977 0.09873319 0.10269022]

linear svm
accuracies= [96.59560929 96.91377665 88.32325803 95.73655743 96.53197582 95.89564111
 97.35921094 96.85014318 96.46834235 96.75469297]
total acc= 95.74 2.51
test time= [0.02594924 0.02695608 0.02593422 0.02593088 0.02489567 0.02596712
 0.02493906 0.02495575 0.02594829 0.02496123]



In [61]:
start_time = time.time()
X=np.array([cnn_model.extract_features(os.path.join(DATASET_PATH,filepath),crop_center=crop_center) for filepath in files4verification])    
print('--- %s seconds ---' % (time.time() - start_time))
print('Verification')

print(X.shape)
X=preprocessing.normalize(X,norm='l2')

print_verification_results(X)

C:\Users\avsavchenko\AppData\Local\Temp\ipykernel_18884\1513849870.py:55: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  x = np.array(Image.fromarray(img).resize((self.w,self.h),resample=Image.BILINEAR)).astype(float)


--- 235.93100428581238 seconds ---
Verification
(7701, 512)
Accuracy: 0.99283+-0.00527
Validation rate: 0.97667+-0.01430 @ FAR=0.00100
Area Under Curve (AUC): 0.99939
Equal Error Rate (EER): 0.008


In [ ]:
cnn_model.close_session()